In [1]:
import sys
!{sys.executable} -m pip install pycaret

In [3]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from pycaret.classification import *


In [4]:
train = pd.read_csv("/Users/keirynhart/Documents/Uni/Data 301/assignment 5/A5_data/train.csv")
test = pd.read_csv("/Users/keirynhart/Documents/Uni/Data 301/assignment 5/A5_data/test.csv")

In [24]:
train_labels = train['label']

test_labels = test["label"]

train_full = train.copy()

train = train.drop(['label'], axis = 1)

test_full = test.copy()

test = test.drop(['label'], axis = 1)

In [ ]:
test.head()

In [5]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors.KNeighborsClassifier,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model.SGDClassifier,True
rbfsvm,SVM - Radial Kernel,sklearn.svm.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process.GPC,False
mlp,MLP Classifier,sklearn.neural_network.MLPClassifier,False
ridge,Ridge Classifier,sklearn.linear_model.RidgeClassifier,True


In [7]:
clf = setup(data = train, target = 'label')

Setup Succesfully Completed!


,Description,Value
0,session_id,3241
1,Target Type,Binary
2,Label Encoded,"negative: 0, positive: 1"
3,Original Data,"(6744, 1025)"
4,Missing Values,False
5,Numeric Features,0
6,Categorical Features,1024
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [12]:
trials = compare_models(sort = 'AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Extra Trees Classifier,0.9352,0.8698,0.3856,0.6940,0.4915,0.4603,0.4850,0.5902
1,CatBoost Classifier,0.9345,0.8674,0.3111,0.7487,0.4368,0.4086,0.4546,30.1576
2,Light Gradient Boosting Machine,0.9352,0.8604,0.3958,0.6878,0.4995,0.4677,0.4896,1.1767
3,Extreme Gradient Boosting,0.9352,0.8532,0.3906,0.6910,0.4946,0.4632,0.4869,34.3761
4,Gradient Boosting Classifier,0.9331,0.8455,0.3084,0.7261,0.4293,0.4000,0.4433,10.0004
5,Logistic Regression,0.9150,0.8218,0.3883,0.4862,0.4305,0.3853,0.3888,0.5021
6,K Neighbors Classifier,0.9314,0.8170,0.3545,0.6604,0.4561,0.4235,0.4492,0.9005
7,Random Forest Classifier,0.9309,0.8116,0.2928,0.7141,0.4104,0.3804,0.4256,0.1462
8,Linear Discriminant Analysis,0.9138,0.8049,0.4756,0.4776,0.4746,0.4279,0.4289,3.2726
9,Ada Boost Classifier,0.9271,0.8011,0.3012,0.6239,0.4015,0.3682,0.3980,3.2257


After using the training set to evaluate a varietly of different machine learning models we can see that based on AUC the top 3 sklearn classifiers are Extra Trees Classifier, Gradient boosting classifier, logistic regression, K Neighbours Classifier and Random forest classifier. because of this these are the 5 classifiers that I am going to continue with. 

In [66]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

In [18]:
xt = ExtraTreesClassifier()
grad = GradientBoostingClassifier()
log = LogisticRegression()
knn = KNeighborsClassifier(n_neighbors=3)
rf = RandomForestClassifier()
classifier_list = [xt,grad,log,knn,rf]

In [60]:
from sklearn.decomposition import PCA

pca = PCA(n_components= 20)
train_pca = pca.fit(train).transform(train)
       
test_pca = pca.transform(test)

def classifiers(train, train_labels, test, test_labels):
    for i in classifier_list:
        
        i.fit(train_pca, train_labels)
        
        pred = i.predict(test_pca)
        pred1 = cross_val_predict(i, train, train_labels, cv = 10)
        
        print(accuracy_score(pred1, train_labels))
        print(classification_report(test_labels, pred))

In [61]:
classifiers(train, train_labels, test, test_labels)

0.9430604982206405
              precision    recall  f1-score   support

    negative       0.94      0.99      0.96      2063
    positive       0.76      0.26      0.39       185

    accuracy                           0.93      2248
   macro avg       0.85      0.63      0.68      2248
weighted avg       0.92      0.93      0.92      2248

0.9337188612099644
              precision    recall  f1-score   support

    negative       0.93      0.99      0.96      2063
    positive       0.69      0.20      0.31       185

    accuracy                           0.93      2248
   macro avg       0.81      0.60      0.64      2248
weighted avg       0.91      0.93      0.91      2248

0.9233392645314353
              precision    recall  f1-score   support

    negative       0.93      1.00      0.96      2063
    positive       0.73      0.10      0.18       185

    accuracy                           0.92      2248
   macro avg       0.83      0.55      0.57      2248
weighted avg     

In [73]:
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold

In [71]:
models = [
    ('xt', ExtraTreesClassifier()),
    ('grad', GradientBoostingClassifier()),
    ('log', LogisticRegression()),
    ('knn', KNeighborsClassifier(n_neighbors=3)),
    ('rf', RandomForestClassifier())
]

In [74]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

In [76]:
for clf in models:
    scores = cross_val_score(clf[1], train, train_labels, scoring='accuracy', cv=cv, n_jobs=-1)
    print('%s: %.4f (%.4f)' % (clf[0], np.mean(scores), np.std(scores)))

xt: 0.9427 (0.0048)
grad: 0.9346 (0.0058)
log: 0.9234 (0.0085)
knn: 0.9290 (0.0071)
rf: 0.9408 (0.0050)


In [85]:
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('xt', ExtraTreesClassifier()))
    level0.append(('grad', GradientBoostingClassifier()))
    level0.append(('log', LogisticRegression()))
    level0.append(('knn', KNeighborsClassifier(n_neighbors=3)))
    level0.append(('rf', RandomForestClassifier()))
    # define meta learner model
    level1 = ExtraTreesClassifier()
    # define the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model

In [ ]:
model = get_stacking()
print(cross_val_score(xt, test, test_labels, scoring="roc_auc", cv = 10).max())

In [100]:
clf = StackingClassifier(estimators=models,
                        final_estimator=ExtraTreesClassifier())
scores = cross_val_score(model, train, train_labels, scoring='accuracy', cv=cv, n_jobs=-1)
print('%s: %.4f (%.4f)' % (name, np.mean(scores), np.std(scores)))

NameError: name 'name' is not defined